In [1]:
import pandas as pd
import json
from tqdm.notebook import tqdm

In [2]:
YEAR_RANGE = [2019, 2020]
df_311 = pd.read_csv('data/raw/311_Block_Group_Processed.csv')
# df_311 = df_311[df_311['CREATION YEAR'].between(YEAR_RANGE[0], YEAR_RANGE[1])]

In [3]:
len(df_311['REQUEST TYPE'].unique())

1229

In [4]:
df_311['CREATION YEAR'].unique()

array([2019, 2020, 2009, 2018, 2016, 2015, 2014, 2007, 2011, 2017, 2013,
       2012, 2010, 2008, 2006, 2021], dtype=int64)

In [5]:
df_solution = df_311[['DEPARTMENT', 'WORK GROUP']]
df_problem = df_311[['REQUEST TYPE', 'CATEGORY', 'TYPE', 'DETAIL']]

df_solution.head()

,DEPARTMENT,WORK GROUP
0,Public Works,Public Works-Street and Traffic-District 1
1,Parks and Rec,Parks and Rec-Central Region-
2,NHS,NHS-Dangerous Buildings-
3,NHS,NHS-Neighborhood Preservation-
4,NHS,NHS-Solid Waste-


In [22]:
df_solution['DEPARTMENT'].drop_duplicates().to_csv('Taxonomy/unique_departments.csv', index = False)
df_solution['WORK GROUP'].drop_duplicates().to_csv('Taxonomy/unique_workgroups.csv', index = False)
df_solution.groupby(['DEPARTMENT', 'WORK GROUP']).count().to_csv('Taxonomy/solution_domain_hierarchy.csv')


In [20]:
df_solution.groupby(['DEPARTMENT', 'WORK GROUP']).count()

Empty DataFrame
Columns: []
Index: [(Aviation, Aviation--), (City Managers Office, City Managers Office-311 Call Center-), (City Managers Office, City Managers Office-311 Call Center-Administratio), (City Managers Office, City Managers Office-311 Call Center-Support), (City Managers Office, City Managers Office-311-Field Staff), (City Managers Office, City Managers Office-Administration-), (City Managers Office, City Managers Office-CIMO-), (City Managers Office, City Managers Office-Capital Projects-), (City Managers Office, City Managers Office-Compass KC-), (City Managers Office, City Managers Office-Emergency Management-), (City Managers Office, City Managers Office-KCPD-), (City Planning and Development, City Planning and Development-Administration), (City Planning and Development, City Planning and Development-Adminstration-), (City Planning and Development, City Planning and Development-Dev Comp-Zoning), (City Planning and Development, City Planning and Development-Elevators-), (City Planning and Development, City Planning and Development-Land Develop-Inspect), (City Planning and Development, City Planning and Development-Permit Compliance-), (Convention and Entertainment Center, Convention and Entertainment Center--), (Finance, Finance--), (Finance, Finance-PIAC-), (Fire, Fire--), (General Service, General Service-City Property-), (General Service, General Service-Facility Management-), (Health, Health--), (Health, Health-Air Quality-), (Health, Health-Communicable Disease Control-), (Health, Health-Community Environmental Health-), (Health, Health-Food Protection-), (Health, Health-Healthy Homes-), (Health, Health-Healthy Homes-Bedbugs), (Health, Health-Lead Poisoning Prevention-), (Health, Health-Noise Control-), (Health, Health-Rat-), (Health, Health-Tobacco Control-), (Housing Community Dev, Housing Community Dev-Property Preservation), (Human Resources, Human Resources--), (IT, IT-testing emails-), (Information Technology, Information Technology--), (Information Technology, Information Technology-ERP Testing-DML), (Information Technology, Information Technology-Email issue-), (KCPD, KCPD-Parking Control-), (Mayors Office, Mayors Office--), (Municipal Court, Municipal Court-Accommodations), (Municipal Court, Municipal Court-Customer Service-), (NCS, NCS-Animal Health and Safety-Jackson County), (NCS, NCS-Neighborhood Preservation-LT List), (NHS, NHS-Administration-), (NHS, NHS-Animal Health and Safety-), (NHS, NHS-Animal Health and Safety-Admin), (NHS, NHS-Dangerous Buildings-), (NHS, NHS-Dangerous Buildings-Emergency), (NHS, NHS-HazardousStructure), (NHS, NHS-NCAP-), (NHS, NHS-NPD-Assessments), (NHS, NHS-Neighborhood Preservation-), (NHS, NHS-Neighborhood Preservation-Abatement), (NHS, NHS-Neighborhood Preservation-Administration), (NHS, NHS-Neighborhood Preservation-DART), (NHS, NHS-Neighborhood Preservation-Illegal Dumping), (NHS, NHS-Neighborhood Preservation-Open Entry), (NHS, NHS-Neighborhood Preservation-Priority), (NHS, NHS-Neighborhood Preservation-Programs), (NHS, NHS-Neighborhood Referrals-Supervisors), (NHS, NHS-Neighborhood Services), (NHS, NHS-Neighborhood-Abatement), (NHS, NHS-Property Preservation-), (NHS, NHS-Regulated Industries-), (NHS, NHS-Salvage-), (NHS, NHS-Solid Waste-), (NHS, NHS-Solid Waste-Abatement Referral), (NHS, NHS-Solid Waste-Administration), (NHS, NHS-Solid Waste-Illegal Dumping), (NHS, NHS-Tow Services-), (NHS, NHS-Tow-Administration), (Northeast, Northeast-Alliance-Together), (Northland, Northland-Neighborhood-Inc), (Parks & Rec, Parks & Rec-Brush Creek-Maintenance), (Parks & Rec, Parks & Rec-Landscape Services-BRWF), (Parks & Rec, Parks & Rec-Landscape Services-Forestry), (Parks & Rec, Parks & Rec-Landscape Services-LT List), (Parks & Rec, Parks & Rec-Landscape Services-ROWWeeds), (Parks & Rec, Parks & Rec-Landscape Services-Visability), (Parks & Rec, Parks & Rec-Planning and Design), (Parks & Recreation, Parks & Recreation-Landscape Services-BRWF), (Parks and Rec, Parks and Rec

In [6]:
department_dict = {x:[] for x in df_solution['DEPARTMENT'].unique().tolist()}
for i, g in df_solution.groupby(['DEPARTMENT','WORK GROUP']):
#     print(i)
    department_dict[i[0]].append(i[1])
#     display(g)
department_dict

{'Public Works': ['Public Works- Sidewalks- GO Bonds',
  'Public Works-Administration-',
  'Public Works-Capital Projects-',
  'Public Works-Capital Projects-Preservation',
  'Public Works-Capital Projects-Sidewalks',
  'Public Works-Capital Projects-Signs',
  'Public Works-Capital Projects-Traffic Control',
  'Public Works-Capital Projects-Traffic Permits',
  'Public Works-Capital Projects-Traffic Signals',
  'Public Works-Coordination Service-Bicycle',
  'Public Works-Facilities Maintenance-',
  'Public Works-Parking Division-',
  'Public Works-Snow and Ice-Administration',
  'Public Works-Solid Waste-',
  'Public Works-Street and Traffic-Administration',
  'Public Works-Street and Traffic-District 1',
  'Public Works-Street and Traffic-District 2',
  'Public Works-Street and Traffic-District 3',
  'Public Works-Street and Traffic-Project Planning',
  'Public Works-Street and Traffic-Rural Mowing',
  'Public Works-Street and Traffic-Snow',
  'Public Works-Street and Traffic-Street Op

In [5]:
'''
Department -> Work Group
'''
hierarchy_obj = [{
    "name": 'Solution Domain',
    "parent": 'null',
    'children': []
    }
]
for k,v in department_dict.items():
    # print(k,v)
    dpt_obj = {
    "name": k,
    "parent": "null",
    "children": []
    }
    '''
    Category -> Type 
    '''
    for cat in v:
        cat_obj = {
            "name": cat,
            "parent": k,
            'children': []
          }
        dpt_obj['children'].append(cat_obj)


    hierarchy_obj[0]['children'].append(dpt_obj)
    # break

with open('departments_workgroup_hierarchy.json', 'w') as f:
    json_obj = json.dumps(hierarchy_obj, indent = 4)
    f.write(json_obj)
print(json_obj)

                   "children": []
                    },
                    {
                        "name": "NHS-Solid Waste-Illegal Dumping",
                        "parent": "NHS",
                        "children": []
                    },
                    {
                        "name": "NHS-Tow Services-",
                        "parent": "NHS",
                        "children": []
                    },
                    {
                        "name": "NHS-Tow-Administration",
                        "parent": "NHS",
                        "children": []
                    }
                ]
            },
            {
                "name": "Water Services",
                "parent": "null",
                "children": [
                    {
                        "name": "Water Services - Wastewater Investigations",
                        "parent": "Water Services",
                        "children": []
                    },
                    {
  

In [16]:
requestType_dict = {x:{} for x in df_problem['REQUEST TYPE'].unique().tolist()}
for i, g in df_problem.groupby(['REQUEST TYPE','CATEGORY', 'TYPE', 'DETAIL']):
    print(i)
    if i[1] not in requestType_dict[i[0]]:
        requestType_dict[i[0]][i[1]] = {i[2]: [i[3]]}
    else: 
        if i[2] not in requestType_dict[i[0]][i[1]]:
            requestType_dict[i[0]][i[1]][i[2]] = [i[3]]
        else:
            requestType_dict[i[0]][i[1]][i[2]].append(i[3])
    # break


ecycling - Leaf/Brush - Thursday - North', 'Trash / Recycling', 'Leaf / Brush', 'Missed Pick Up')
('Trash/Recycling - Leaf/Brush - Thursday - North', 'Trash / Recycling', 'Leaf / Brush', 'Thursday - North')
('Trash/Recycling - Leaf/Brush - Thursday - South', 'Trash / Recycling', 'Leaf / Brush', 'Thursday - South')
('Trash/Recycling - Leaf/Brush - Tuesday - Central', 'Trash / Recycling', 'Leaf / Brush', 'Tuesday - Central')
('Trash/Recycling - Leaf/Brush - Tuesday - North', 'Trash / Recycling', 'Leaf / Brush', 'Monday - Central')
('Trash/Recycling - Leaf/Brush - Tuesday - North', 'Trash / Recycling', 'Leaf / Brush', 'Tuesday - North')
('Trash/Recycling - Leaf/Brush - Tuesday - South', 'Trash / Recycling', 'Leaf / Brush', 'Tuesday - South')
('Trash/Recycling - Leaf/Brush - Wednesday - Central', 'Trash / Recycling', 'Leaf / Brush', 'Wednesday - Central')
('Trash/Recycling - Leaf/Brush - Wednesday - North', 'Trash / Recycling', 'Leaf / Brush', 'Wednesday - North')
('Trash/Recycling - Leaf/

In [7]:
category_dict['Trash / Recycling']

NameError: name 'category_dict' is not defined

In [ ]:
print(requestType_dict)
# print(category_dict)
# print(type_dict)

In [17]:
len(requestType_dict)

1229

In [25]:
'''
Request Type -> Category -> Type -> Detail Hierarchy
'''
TYPE_MATCH = 'Bulky Pick Up', 'Dumping', 'Dumpsters', 'Early Set Out'

hierarchy_obj = [{
    "name": 'Problem Domain',
    "parent": 'null',
    'children': []
    }
]
count = 0
total_count_dict = {
    'category': 0,
    'type': 0,
    'detail': 0,
    'request_type': 0
}
for k,category_dict in requestType_dict.items():
    # if count >= 20:
    #     break
    # print(k,v)
    rqType_obj = {
        "name": k,
        "parent": "null",
        "children": []
    }
    '''
    Category -> Type 
    '''
    for cat, type_dict in category_dict.items():
        if cat != 'Trash / Recycling':
            break
        cat_obj = {
            "name": cat,
            "parent": k,
            'children': []
          }
        '''
        Type -> Detail 
        '''
        for typ, details in type_dict.items():
            if typ not in TYPE_MATCH:
                continue
            type_obj = {
            "name": typ,
            "parent": cat,
            'children': []
            }
            for detail in details:
                det_obj = {
                    "name": detail,
                    "parent": typ,
                }
                type_obj['children'].append(det_obj)
                total_count_dict['detail'] += 1

            cat_obj['children'].append(type_obj)
            total_count_dict['type'] += 1
        if len(cat_obj['children']) > 0:
            rqType_obj['children'].append(cat_obj)
        total_count_dict['category'] += 1

    if len(rqType_obj['children']) > 0:
        hierarchy_obj[0]['children'].append(rqType_obj)
    count+=1
    total_count_dict['request_type'] += 1 
    # break

with open('temp3.json', 'w') as f:
    json_obj = json.dumps(hierarchy_obj, indent = 4)
    f.write(json_obj)
print(json_obj)

                        {
                                "name": "Dumping",
                                "parent": "Trash / Recycling",
                                "children": [
                                    {
                                        "name": "Solid Waste Referral",
                                        "parent": "Dumping"
                                    }
                                ]
                            }
                        ]
                    }
                ]
            },
            {
                "name": "Trash / Recycling-Dumping-ROW North",
                "parent": "null",
                "children": [
                    {
                        "name": "Trash / Recycling",
                        "parent": "Trash / Recycling-Dumping-ROW North",
                        "children": [
                            {
                                "name": "Dumping",
                                "parent": "Trash / Re

In [20]:
total_count_dict

{'category': 1349, 'type': 1401, 'detail': 1537, 'request_type': 1229}

In [6]:
'''
Ordered by Category first
'''
category_dict = {x:{} for x in df_problem['CATEGORY'].unique().tolist()}

for i, g in df_problem.groupby(['CATEGORY','TYPE', 'DETAIL', 'REQUEST TYPE']):
    # print(i)
    if i[1] not in category_dict[i[0]]:
        category_dict[i[0]][i[1]] = {i[2]: [i[3]]}
    else: 
        if i[2] not in category_dict[i[0]][i[1]]:
            category_dict[i[0]][i[1]][i[2]] = [i[3]]
        else:
            category_dict[i[0]][i[1]][i[2]].append(i[3])
    # break
#     display(g)
# requestType_dict

In [9]:
df_problem.groupby(['CATEGORY','TYPE', 'DETAIL', 'REQUEST TYPE']).count()

Empty DataFrame
Columns: []
Index: [(Air Quality, Pollutants, Indoor, Healthy Homes Program), (Air Quality, Pollutants, Outdoor, Outdoor Air Quality), (Air Quality, Tobacco, All, Smoking Complaints), (Animal, Barking Dog, All, Barking Dog), (Animal, Bite, *Select one, Animal Bite), (Animal, Bite, All, Animal Bite), (Animal, Emergency, *Select one, Animal Emergency), (Animal, Emergency, Abused, Animal Emergency), (Animal, Emergency, Aggressive, Animal Emergency), (Animal, Emergency, Bat visible in house, Animal Control), (Animal, Emergency, Bat visible in house, Animal Emergency), (Animal, Emergency, Injured Cat or Dog, Animal Emergency), (Animal, Injury or Cruelty involving an Animal, All, Injury or Cruelty Involving an Animal), (Animal, Insect Problem, Bedbugs, Bedbugs Problem), (Animal, Insect Problem, Integrated Pest Management Class, Integrated Pest Management Class), (Animal, Investigation, All, Animal Investigation), (Animal, Jackson County, Buckner, Animal Concerns Buckner Jackson County), (Animal, Jackson County, Buckner, Animal Emergency Buckner Jackson County), (Animal, Jackson County, Greenwood, Animal Concerns Greenwood Jackson County), (Animal, Jackson County, Greenwood, Animal Emergency Greenwood Jackson County), (Animal, Jackson County, Levacy, Animal Concerns Levacy Jackson County), (Animal, Jackson County, Levacy, Emergency Animal Levacy Jackson County), (Animal, Jackson County, Lone Jack, Animal Concerns Lone Jack Jackson County), (Animal, Jackson County, Lone Jack, Animal Emergency Lone Jack Jackson County), (Animal, Jackson County, Sibley, Animal Concerns Sibley Jackson County), (Animal, Jackson County, Sibley, Animal Emergency Greenwood Jackson County), (Animal, Jackson County, Sibley, Animal Emergency Sibley Jackson County), (Animal, Jackson County, Sugar Creek, Animal Concerns Sugar Creek Jackson County), (Animal, Jackson County, Sugar Creek, Animal Emergency Sugar Creek Jackson County), (Animal, Jackson County, Unincorporated Areas of Jackson County, Animal Concerns Unincorporated Jackson County), (Animal, Jackson County, Unincorporated Areas of Jackson County, Animal Concers Unicorporated Jackson County), (Animal, Jackson County, Unincorporated Areas of Jackson County, Animal Emergency Unincorporated Jackson County), (Animal, Jackson County, Unity Village, Animal Concerns Unity Village Jackson County), (Animal, Jackson County, Unity Village, Animal Emergency Unity Village Jackson County), (Animal, Other, Non Emergency, Animal Control), (Animal, Other, Non Emergency, Animal Investigation), (Animal, Owned or Stray at Large, All, Animal At Large), (Animal, Repair, *Select one, Animal Control), (Animal, Return Call to Citizen, All, Animal Admin Call Back), (Animal, Return Call to Citizen, All, Animal Control Administration), (Animal, Return Call to Citizen, Jack, Rat Control Education), (Animal, Rodent, Rat, Rat Control Education), (Animal, Rodent, Rat Control Treatment and Baiting, Rat Control Treatment), (Animal, Stray Confined, Confined, Stray Animal Confined), (Animal, Wildlife, All, Wildlife), (Animals / Pets, Bite, Animal on Animal, Animals / Pets-Bite-Animal on Animal), (Animals / Pets, Bite, Owned, Animals / Pets-Bite-Owned), (Animals / Pets, Bite, Stray, Animals / Pets-Bite-Stray), (Animals / Pets, Bite, Wildlife, Animals / Pets-Bite-Wildlife), (Animals / Pets, Cruelty or Neglect, Abandonment, Animals / Pets-Cruelty or Neglect-Abandonment), (Animals / Pets, Cruelty or Neglect, Abuse / Injury / Death, Animals / Pets-Cruelty or Neglect-Abuse / Injury / Death), (Animals / Pets, Cruelty or Neglect, Inclement Weather, Animals / Pets-Cruelty or Neglect-Inclement Weather), (Animals / Pets, Cruelty or Neglect, Shelter / Food / Water, Animals / Pets-Cruelty or Neglect-Shelter / Food / Water), (Animals / Pets, Dead Animal, Curb, Animals / Pets-Dead Animal -Curb), (Animals / Pets, Dead Animal, Private Property Nuisance, Property Violations), (Animals / Pets, Dead Animal, Private Property Pickup, Animals / Pe

In [15]:
'''
Print out Hierarchy in CSV
'''
df_problem.groupby(['CATEGORY','TYPE', 'DETAIL', 'REQUEST TYPE']).count().to_csv('Taxonomy/hierarchy.csv', index = True)

In [11]:
frequency_table = df_problem.groupby(['CATEGORY','TYPE', 'DETAIL', 'REQUEST TYPE']).size().to_frame()
# display(frequency_table)
# frequency_table.sum(level = 'TYPE')
frequency_table.xs('Trash / Recycling', level='CATEGORY', drop_level=False)


0
CATEGORY          TYPE             DETAIL                            REQUEST TYPE                                             
Trash / Recycling Bulky Pick Up    Address Validation                Trash / Recycling-Bulky Pick Up -Address Valida...    491
                                   Missed Pick Up                    Trash / Recycling-Bulky Pick Up -Missed Pick Up      7707
                  Dumping          Alley - Public Works - District 1 Trash / Recycling - Dumping - Alley - D1                3
                                                                     Trash / Recycling-Dumping-Alley                       253
                                   Alley - Public Works - District 2 Trash / Recycling - Dumping - Alley - D2              808
...                                                                                                                        ...
                  Trash Collection Missed by City                    Trash / Recycling-Trash Collection-Missed by City   17491
                                   Missed by Contractor North        Trash / Recycling-Trash Collection-Missed by Co...  21681
                                   Missed by Contractor South        Trash / Recycling-Trash Collection-Missed by Co...  27538
                                   Repeat Missed                     Trash / Recycling-Trash Collection-Repeat Missed      871
                                   Storm Trash                       Trash/Recycling - Trash Collection - Storm Trash       12

[101 rows x 1 columns]

In [9]:
len(category_dict)

84

In [111]:
cat_freq_df[cat_freq_df.index == k].values[0][0]

199720

In [13]:
'''
Category -> Type -> Detail -> Request Type Hierarchy (Relative Frequency)
'''
import random
hierarchy_obj = [{
    "name": 'Problem Domain',
    "parent": 'null',
    'children': []
    }
]
category_count = 0
max_types = random.randint(2,5)
max_details = random.randint(2,5)
cat_freq_df = frequency_table.sum(level='CATEGORY')
total_cat = cat_freq_df.sum().values[0]

total_count_dict = {
    'category': 0,
    'type': 0,
    'detail': 0,
    'request_type': 0
}
# TYPE_MATCH = 'Bulky Pick Up', 'Dumping', 'Dumpsters', 'Early Set Out'
for k,type_dict in tqdm(category_dict.items()):
    # if category_count >= 5:
    #     break
    # if k != 'Trash / Recycling':
    #     continue
    
    cat_sum = cat_freq_df[cat_freq_df.index == k].values[0][0]
    cat_freq = round(cat_sum/total_cat,4)
    new_cat = "{} - {} - {}".format(k, str(cat_sum), str(cat_freq))
    # print(cat_sum)
    type_count = 0
    cat_obj = {
    "name": new_cat,
    "parent": "null",
    "children": []
    }
    '''
    Calculate Type Frequency
    '''
    type_freq_df = frequency_table.xs(k, level='CATEGORY', drop_level=False).sum(level = 'TYPE')
    total_type = type_freq_df.sum().values[0]
    '''
    Category -> Type 
    '''
    for typ, detail_dict in type_dict.items():
        # if type_count >= max_types:
        #     break
        # if typ not in TYPE_MATCH:
        #     continue
        detail_count = 0
        type_sum = type_freq_df[type_freq_df.index == typ].values[0][0]
        type_freq = round(type_sum/total_type, 4)

        # print(type_freq)
        new_typ = "{} - {} - {}".format(typ, str(type_sum), str(type_freq))
        type_obj = {
            "name": new_typ,
            "parent": k,
            'children': []
          }

        '''
        Calculate Detail Frequency
        '''
        detail_freq_df = frequency_table.xs(k, level='CATEGORY', drop_level=False).xs(typ, level='TYPE', drop_level=False).sum(level = 'DETAIL')
        total_detail = detail_freq_df.sum().values[0]
        '''
        Type -> Detail 
        '''
        for detail, request_types in detail_dict.items():
            # if detail_count > max_details: 
            #     break

            detail_sum = detail_freq_df[detail_freq_df.index == detail].values[0][0]
            detail_freq = round(detail_sum/total_detail, 4)
            new_detail = "{} - {} - {}".format(detail, str(detail_sum), str(detail_freq))
            detail_obj = {
            "name": new_detail,
            "parent": new_typ,
            'children': []
            }

            reqType_freq_df = frequency_table.xs(k, level='CATEGORY', drop_level=False).xs(typ, level='TYPE', drop_level=False).xs(detail, level='DETAIL', drop_level=False).sum(level = 'REQUEST TYPE')
            total_reqType = reqType_freq_df.sum().values[0]
            for req_type in request_types:
                reqType_sum = reqType_freq_df[reqType_freq_df.index == req_type].values[0][0]
                reqType_freq = round(reqType_sum/total_reqType, 4)
                new_reqType= "{} - {} - {}".format(req_type, str(reqType_sum),str(reqType_freq))
                request_obj = {
                    "name": new_reqType,
                    "parent": new_detail,
                }
                detail_obj['children'].append(request_obj)
                total_count_dict['request_type'] += 1
            type_obj['children'].append(detail_obj)
            detail_count+=1
            total_count_dict['detail'] += 1
        type_count+=1
        total_count_dict['type'] += 1
        cat_obj['children'].append(type_obj)


    hierarchy_obj[0]['children'].append(cat_obj)
    category_count+=1
    total_count_dict['category'] += 1
    # break


# with open('temp1.json', 'w') as f:
#     json_obj = json.dumps(hierarchy_obj, indent = 4)
#     f.write(json_obj)
# print(json_obj)

In [14]:
total_count_dict

{'category': 84, 'type': 366, 'detail': 1106, 'request_type': 1537}

In [15]:
'''
Category -> Type -> Detail -> Request Type Hierarchy (Absolute Frequency)
'''
import random
hierarchy_obj = [{
    "name": 'Problem Domain',
    "parent": 'null',
    'children': []
    }
]
category_count = 0
max_types = random.randint(2,5)
max_details = random.randint(2,5)
cat_freq_df = frequency_table.sum(level='CATEGORY')
total_cat = cat_freq_df.sum().values[0]
# TYPE_MATCH = 'Bulky Pick Up', 'Dumping', 'Dumpsters', 'Early Set Out'
total_count_dict = {
    'category': 0,
    'type': 0,
    'detail': 0,
    'request_type': 0
}
for k,type_dict in tqdm(category_dict.items()):
    # if category_count >= 5:
    #     break
    # if k != 'Trash / Recycling':
    #     continue
    
    cat_sum = cat_freq_df[cat_freq_df.index == k].values[0][0]
    cat_freq = round(cat_sum/total_cat,4)
    new_cat = "{} - {} - {}".format(k, str(cat_sum), str(cat_freq))
    # print(cat_freq)
    type_count = 0
    cat_obj = {
    "name": new_cat,
    "parent": "null",
    "children": []
    }
    '''
    Calculate Type Frequency
    '''
    type_freq_df = frequency_table.xs(k, level='CATEGORY', drop_level=False).sum(level = 'TYPE')
    total_type = type_freq_df.sum().values[0]
    '''
    Category -> Type 
    '''
    for typ, detail_dict in type_dict.items():
        # if type_count >= max_types:
        #     break
        # if typ not in TYPE_MATCH:
        #     continue
        detail_count = 0
        type_sum = type_freq_df[type_freq_df.index == typ].values[0][0]
        type_freq = round((type_sum/total_type) * cat_freq, 4)
        new_typ = "{} - {} - {}".format(typ, str(type_sum), str(type_freq))
        type_obj = {
            "name": new_typ,
            "parent": k,
            'children': []
          }

        '''
        Calculate Detail Frequency
        '''
        detail_freq_df = frequency_table.xs(k, level='CATEGORY', drop_level=False).xs(typ, level='TYPE', drop_level=False).sum(level = 'DETAIL')
        total_detail = detail_freq_df.sum().values[0]
        '''
        Type -> Detail 
        '''
        for detail, request_types in detail_dict.items():
            # if detail_count > max_details: 
            #     break

            detail_sum = detail_freq_df[detail_freq_df.index == detail].values[0][0]
            detail_freq = round((detail_sum/total_detail) * type_freq, 4)
            new_detail = "{} - {} - {}".format(detail, str(detail_sum), str(detail_freq))
            detail_obj = {
            "name": new_detail,
            "parent": new_typ,
            'children': []
            }

            reqType_freq_df = frequency_table.xs(k, level='CATEGORY', drop_level=False).xs(typ, level='TYPE', drop_level=False).xs(detail, level='DETAIL', drop_level=False).sum(level = 'REQUEST TYPE')
            total_reqType = reqType_freq_df.sum().values[0]
            for req_type in request_types:
                reqType_sum = reqType_freq_df[reqType_freq_df.index == req_type].values[0][0]
                reqType_freq = round((reqType_sum/total_reqType) * detail_freq, 4)
                new_reqType= "{} - {} - {}".format(req_type, str(reqType_sum),str(reqType_freq))
                request_obj = {
                    "name": new_reqType,
                    "parent": new_detail,
                }
                detail_obj['children'].append(request_obj)
                total_count_dict['request_type'] += 1

                
            type_obj['children'].append(detail_obj)
            total_count_dict['detail'] += 1
            detail_count+=1
        type_count+=1
        total_count_dict['type'] += 1

        cat_obj['children'].append(type_obj)


    hierarchy_obj[0]['children'].append(cat_obj)
    category_count+=1
    total_count_dict['category'] += 1
    # break


# with open('temp2.json', 'w') as f:
#     json_obj = json.dumps(hierarchy_obj, indent = 4)
#     f.write(json_obj)
# print(json_obj)

In [144]:
frequency_table.columns=['Count']

In [166]:
# frequency_table.xs(['Trash / Recycling'], level='CATEGORY', drop_level=False)
temp = frequency_table.iloc[frequency_table.index.get_level_values('TYPE').isin(TYPE_MATCH)]
display(temp)
# temp.to_csv('Taxonomy/category_counts-subset.csv')


0
CATEGORY          TYPE          DETAIL                            REQUEST TYPE                                            
Trash / Recycling Bulky Pick Up Address Validation                Trash / Recycling-Bulky Pick Up -Address Valida...   327
                                Missed Pick Up                    Trash / Recycling-Bulky Pick Up -Missed Pick Up     2047
                  Dumping       Alley - Public Works - District 1 Trash / Recycling - Dumping - Alley - D1               1
                                Alley - Public Works - District 2 Trash / Recycling - Dumping - Alley - D2             333
                                Alley - Public Works - District 3 Trash / Recycling - Dumping - Alley - D3              30
                                Camera Request                    Trash / Recycling-Dumping-Camera Request             202
                                Landlord Set Out                  Trash / Recycling-Dumping-Landlord Set Out             2
                                Park - Central                    Trash / Recycling-Dumping-Park - Central             108
                                Park - North                      Trash / Recycling-Dumping-Park - North                27
                                Park - South                      Trash / Recycling-Dumping-Park - South                49
                                Private Property                  Trash / Recycling-Dumping-Private Property           648
                                Right of Way Central              Trash / Recycling-Dumping-ROW Central               1828
                                Right of Way LB Central           Trash / Recycling-Dumping-ROW Land Bank Central      352
                                Right of Way LB North             Trash / Recycling-Dumping-ROW Land Bank North          3
                                Right of Way LB South             Trash / Recycling-Dumping-ROW Land Bank South         51
                                Right of Way North                Trash / Recycling-Dumping-ROW North                  300
                                Right of Way South                Trash / Recycling-Dumping-ROW South                  621
                                Right-Of-Way                      Trash / Recycling-Dumping-Right of Way                 8
                                Solid Waste Referral              Trash / Recycling-Dumping-Solid Waste Referral      3590
                  Dumpsters     Dumpster Request                  Trash / Recycling-Dumpsters-Dumpster Request           7
                                Private Property                  Property Violations                                   44
                                Street / Alley / ROW              Trash / Recycling-Dumpsters-Street / Alley / ROW      39
                  Early Set Out Central                           Trash / Recycling-Early Set Out-Central              108
                                North                             Trash / Recycling-Early Set Out-North                 20
                                South                             Trash / Recycling-Early Set Out-South                 46

<h1> Category - Department taxonomy

In [57]:
df_inconsistency = df_311.copy().loc[:, ['CATEGORY', 'DEPARTMENT']]
df_inconsistency.head()

,CATEGORY,DEPARTMENT
0,Streets / Roadways / Alleys,Public Works
1,Parks & Recreation,Parks and Rec
2,Property / Buildings / Construction,NHS
3,Property / Buildings / Construction,NHS
4,Trash / Recycling,NHS


In [58]:
inconsistency_dict = {x:[] for x in df_inconsistency['CATEGORY'].unique().tolist()}
for i, g in df_inconsistency.groupby(['CATEGORY','DEPARTMENT']):
#     print(i)
    inconsistency_dict[i[0]].append(i[1])
#     display(g)
inconsistency_dict

{'Streets / Roadways / Alleys': ['City Managers Office',
  'City Planning and Development',
  'KCPD',
  'NHS',
  'Parks and Rec',
  'Public Works',
  'Water Services'],
 'Parks & Recreation': ['City Managers Office',
  'Information Technology',
  'NHS',
  'Parks & Rec',
  'Parks and Rec',
  'Public Works',
  'Water Services'],
 'Property / Buildings / Construction': ['City Managers Office',
  'City Planning and Development',
  'General Service',
  'Health',
  'KCPD',
  'NHS',
  'Northeast',
  'Northland',
  'Public Works',
  'South'],
 'Trash / Recycling': ['City Managers Office',
  'NHS',
  'Northland',
  'Parks and Rec',
  'Public Works',
  'Water Services'],
 'Trees': ['City Managers Office',
  'NHS',
  'Northland',
  'Parks & Rec',
  'Parks and Rec',
  'Water Services'],
 'Water': ['City Managers Office',
  'Health',
  'Parks and Rec',
  'Public Works',
  'Water Services'],
 'Lights / Signals': ['City Managers Office',
  'City Planning and Development',
  'NHS',
  'Parks and Rec',


In [62]:
'''
Category -> Department
'''
hierarchy_obj = [{
    "name": 'Inconsistencies',
    "parent": 'null',
    'children': []
    }
]
MAX_CAT = 5
count = 0
for k,v in inconsistency_dict.items():
    # print(k,v)
    if count >= MAX_CAT:
        break
    cat_obj = {
    "name": k,
    "parent": "null",
    "children": []
    }
    '''
    Category -> Type 
    '''
    for dpt in v:
        dpt_obj = {
            "name": dpt,
            "parent": k,
            'children': []
          }
        cat_obj['children'].append(dpt_obj)


    hierarchy_obj[0]['children'].append(cat_obj)
    # break
    count+=1
with open('inconsistency_hierarchy.json', 'w') as f:
    json_obj = json.dumps(hierarchy_obj, indent = 4)
    f.write(json_obj)
print(json_obj)

[
    {
        "name": "Inconsistencies",
        "parent": "null",
        "children": [
            {
                "name": "Streets / Roadways / Alleys",
                "parent": "null",
                "children": [
                    {
                        "name": "City Managers Office",
                        "parent": "Streets / Roadways / Alleys",
                        "children": []
                    },
                    {
                        "name": "City Planning and Development",
                        "parent": "Streets / Roadways / Alleys",
                        "children": []
                    },
                    {
                        "name": "KCPD",
                        "parent": "Streets / Roadways / Alleys",
                        "children": []
                    },
                    {
                        "name": "NHS",
                        "parent": "Streets / Roadways / Alleys",
                        "children": []
   

<h1> Redundancy Taxonomy

In [66]:
from tqdm.notebook import tqdm
df_redundancy = df_311.copy().loc[:, ['REQUEST TYPE', 'CATEGORY', 'TYPE', 'DETAIL']]

In [67]:
redundancy_list = []
for index, row in tqdm(df_redundancy.iterrows(), total = df_redundancy.shape[0]):
    if row['REQUEST TYPE'] == row['CATEGORY']:
        if row['CATEGORY'] not in redundancy_list:
            redundancy_list.append(row['CATEGORY'])
redundancy_list


['Property Violations',
 'Nuisance Violations',
 'Water Main Break',
 'Bulky Appointment']

In [70]:
df_redundancy = df_redundancy[df_redundancy['REQUEST TYPE'].isin(redundancy_list)]


In [71]:
redundancy_dict = {x:[] for x in df_redundancy['REQUEST TYPE'].unique().tolist()}
for i, g in df_redundancy.groupby(['REQUEST TYPE','CATEGORY']):
#     print(i)
    redundancy_dict[i[0]].append(i[1])
#     display(g)
redundancy_dict

{'Property Violations': ['Animals / Pets',
  'Data Not Available',
  'Mowing / Weeds',
  'Neighborhood Preservation',
  'Property / Buildings / Construction',
  'Property Violations',
  'Sidewalks / Curbs / Ditch',
  'Storm Water / Sewer',
  'Trash / Recycling',
  'Trees',
  'Vehicles / Parking'],
 'Nuisance Violations': ['Data Not Available',
  'Graffiti',
  'Nuisance Violations'],
 'Bulky Appointment': ['Bulky Appointment', 'Trash'],
 'Water Main Break': ['Water Main Break']}

In [72]:
'''
REQUEST TYPE -> CATEGORY (REDUNDANCY)
'''
hierarchy_obj = [{
    "name": 'Redundancy',
    "parent": 'null',
    'children': []
    }
]
MAX_CAT = 5
count = 0
for k,v in redundancy_dict.items():
    # print(k,v)
    reqType_obj = {
    "name": k,
    "parent": "null",
    "children": []
    }
    for cat in v:
        cat_obj = {
            "name": cat,
            "parent": k,
            'children': []
          }
        reqType_obj['children'].append(cat_obj)


    hierarchy_obj[0]['children'].append(reqType_obj)
    # break
    count+=1
with open('redundancy_hierarchy.json', 'w') as f:
    json_obj = json.dumps(hierarchy_obj, indent = 4)
    f.write(json_obj)
print(json_obj)

[
    {
        "name": "Redundancy",
        "parent": "null",
        "children": [
            {
                "name": "Property Violations",
                "parent": "null",
                "children": [
                    {
                        "name": "Animals / Pets",
                        "parent": "Property Violations",
                        "children": []
                    },
                    {
                        "name": "Data Not Available",
                        "parent": "Property Violations",
                        "children": []
                    },
                    {
                        "name": "Mowing / Weeds",
                        "parent": "Property Violations",
                        "children": []
                    },
                    {
                        "name": "Neighborhood Preservation",
                        "parent": "Property Violations",
                        "children": []
                    },
          

<h1> 311 Vacant Calls

In [94]:
df_Vacant311 = df_311.copy().loc[:, ['REQUEST TYPE', 'CATEGORY', 'TYPE', 'DETAIL']]
df_Vacant311 =df_Vacant311[df_Vacant311.apply(lambda col: col.str.contains('Vacant', na=False), axis=1).any(1)]

df_Vacant311

,REQUEST TYPE,CATEGORY,TYPE,DETAIL
145,Property Violations,Mowing / Weeds,Private / Commercial,Lot - Vacant
159,Property Violations,Mowing / Weeds,Private / Commercial,Lot - Vacant
428,Property Violations,Mowing / Weeds,Private / Commercial,Lot - Vacant
555,Property Violations,Mowing / Weeds,Private / Commercial,Lot - Vacant
556,Property Violations,Mowing / Weeds,Private / Commercial,Lot - Vacant
...,...,...,...,...
1543219,Property Violations,Mowing / Weeds,Private / Commercial,Lot - Vacant
1544128,Property Violations,Mowing / Weeds,Private / Commercial,Lot - Vacant
1544647,Property Violations,Mowing / Weeds,Private / Commercial,Lot - Vacant
1544741,Property Violations,Mowing / Weeds,Private / Commercial,Lot - Vacant


In [95]:
requestType_dict = {x:[] for x in df_PropVlt['REQUEST TYPE'].unique().tolist()}
for i, g in df_PropVlt.groupby(['REQUEST TYPE','CATEGORY']):
#     print(i)
    requestType_dict[i[0]].append(i[1])
#     display(g)
category_dict = {x:[] for x in df_PropVlt['CATEGORY'].unique().tolist()}
for i, g in df_PropVlt.groupby(['CATEGORY','TYPE']):
#     print(i)
    category_dict[i[0]].append(i[1])
#     display(g)
    
type_dict = {x:[] for x in df_PropVlt['TYPE'].unique().tolist()}
for i, g in df_PropVlt.groupby(['TYPE','DETAIL']):
#     print(i)
    type_dict[i[0]].append(i[1])
#     display(g)


In [96]:
'''
Request Type -> Category -> Type -> Detail Hierarchy
'''
hierarchy_obj = [{
    "name": '311 Vacant Calls',
    "parent": 'null',
    'children': []
    }
]

for k,v in requestType_dict.items():
    # print(k,v)
    rqType_obj = {
    "name": k,
    "parent": "null",
    "children": []
    }
    '''
    Category -> Type 
    '''
    for cat in v:
        cat_obj = {
            "name": cat,
            "parent": k,
            'children': []
          }
        '''
        Type -> Detail 
        '''
        for typ in category_dict[cat]:
            type_obj = {
            "name": typ,
            "parent": cat,
            'children': []
            }
            for detail in type_dict[typ]:
                det_obj = {
                    "name": detail,
                    "parent": typ,
                }
                type_obj['children'].append(det_obj)
            cat_obj['children'].append(type_obj)

        rqType_obj['children'].append(cat_obj)


    hierarchy_obj[0]['children'].append(rqType_obj)
    # break

with open('311_vacant_hierarchy.json', 'w') as f:
    json_obj = json.dumps(hierarchy_obj, indent = 4)
    f.write(json_obj)
print(json_obj)

[
    {
        "name": "311 Vacant Calls",
        "parent": "null",
        "children": [
            {
                "name": "Property Violations",
                "parent": "null",
                "children": [
                    {
                        "name": "Mowing / Weeds",
                        "parent": "Property Violations",
                        "children": [
                            {
                                "name": "Private / Commercial",
                                "parent": "Mowing / Weeds",
                                "children": [
                                    {
                                        "name": "Lot - Vacant",
                                        "parent": "Private / Commercial"
                                    }
                                ]
                            },
                            {
                                "name": "Public Property",
                                "parent": "Mowing